## AGE DETECTION

### Data Preparation

In [8]:
import os
import numpy as np

In [2]:
faces_path = '/home/falcon-puneet/Documents/College/MinorProject/age_detection/age_gender_race/'
faces_image_names = os.listdir(faces_path)

In [3]:
print(faces_image_names[0])#name of image => age , gender , race , date
len(faces_image_names) #no of image 

65_0_0_20170103183632050.jpg.chip.jpg


23708

In [6]:
def label_name_split(image_name): #functions to split the name into labels
    image_labels = image_name.split('_')
    age = image_labels[0]
    gender = image_labels[1]
    race = image_labels[2]
    return age, gender, race

print(label_name_split(faces_image_names[0]))

('65', '0', '0')


In [11]:
#dictionary to map image name to their labels
faces_age_labels = np.array([])

for image in faces_image_names:
    age, gender, race = label_name_split(image)
    faces_age_labels = np.append(faces_age_labels,age)

In [26]:
unique_ages , unique_ages_count = np.unique(faces_age_labels,return_counts=True) #unqiue age and how many images are there

In [40]:
#dictionary ages and how many image of that image
ages_count = {}
for age,count in zip(unique_ages,unique_ages_count):
    ages_count[age] = count
images_df = pd.DataFrame(ages_count.values(), index=ages_count.keys(), columns=['images'])
print(images_df.head)

<bound method NDFrame.head of      images
1      1123
2       482
3       289
4       273
5       196
..      ...
105       5
110       5
111       1
115       3
116       4

[104 rows x 1 columns]>


## label for age

Age-ranges (classes)<br>
0  1 - 4	
1   5 - 18	
2	19 - 24	
3	25 - 26	
4	27 - 30	
5	31 - 35	
6	36 - 42	
7	43 - 53	
8	54 - 65	
9	66 - 116

In [33]:
import pandas as pd

In [41]:
# Defining a function to split the images in each dataset into n classes (user input).
# Function INPUT: Dataset series of no. of images per age index, and n_classes. 
# Function OUTPUT: Dataset broken up into n_classes with age-ranges, no. of images and proportion balance per class.

def split_classes(ser, n_classes):

    # Calculating the target number of images per class depending on the total no. of images in the dataset.
    n_images = int(sum(ser) / n_classes)

    print(f"Total no. of images in dataset\t= {sum(ser)}")
    print(f"No. of classes desired\t\t= {n_classes}")
    print(f"So, target no. of images/class\t>= {sum(ser)}/{n_classes} = ~{n_images}")
    print()

    # Initiating a dataframe to show the breakdown of age-ranges as output.
    classes_df = pd.DataFrame(columns=['Age-ranges (classes)', 'No. of images', 'Class balance (%)'])

    # Initiating an age index variable to be used to iterate through the ages in the given input series.
    age_index = 0

    for i in range(n_classes):

        # Storing the starting age of the class in a variable age_start.
        # Storing the current age being iterated in a variable age_current.
        # Keeping track of age_index variable so as not to let it go out of index.
        if age_index<=103:
            age_start = ser.index[age_index]
            age_current = ser.index[age_index]
        else:
            break

        # Initiating a new variable to keep track of no. of images added to current class.
        class_images = 0
        
        # Iterating through the ages in the given input series and adding up the no. of images
        # until it exceeds the target number of images per class, using the age_index and age_current variables.
        while class_images < n_images:
            class_images += ser[age_current]
            age_index += 1

            # Keeping track of age_index variable so as not to let it go out of index.
            if age_index<=103:
                age_current = ser.index[age_index]
            else:
                break

        # Storing the ending age of the class in a variable age_end.
        # Keeping track of age_index variable so as not to let it go out of index.
        if age_index<=104:
            age_end = ser.index[age_index-1]
        else:
            break
        
        # Adding the above calculated variables into the dataframe for easier printing and analysis.
        classes_df.loc[i, 'Age-ranges (classes)'] = str(age_start)+" - "+str(age_end)
        classes_df.loc[i, 'No. of images'] = class_images
        classes_df.loc[i, 'Class balance (%)'] = round((class_images / sum(ser)) * 100, 2)
    
    # Calculating some basic statistics about no. of images and the class balance.
    mean_images = int(round(np.mean(classes_df.loc[:, 'No. of images'])))
    mean_balance = round(np.mean(classes_df.loc[:, 'Class balance (%)']), 2)
    std_balance = round(np.std(classes_df.loc[:, 'Class balance (%)']), 2)
    
    print(f"Mean no. of images/class\t= ~{mean_images}")
    print(f"Mean class balance\t\t= {mean_balance}%")
    print(f"Std. of class balance\t\t= {std_balance}%")
    print()

    # Returning the dataframe with all the classes info.
    return classes_df
print(split_classes(images_df['images'],11))

Total no. of images in dataset	= 23708
No. of classes desired		= 11
So, target no. of images/class	>= 23708/11 = ~2155

Mean no. of images/class	= ~2371
Mean class balance		= 10.0%
Std. of class balance		= 1.36%

  Age-ranges (classes) No. of images Class balance (%)
0                1 - 4          2167              9.14
1               5 - 18          2328              9.82
2              19 - 24          2408             10.16
3              25 - 26          2931             12.36
4              27 - 30          2827             11.92
5              31 - 35          2446             10.32
6              36 - 42          2291              9.66
7              43 - 53          2313              9.76
8              54 - 65          2298              9.69
9             66 - 116          1699              7.17


### Data Augmentation